Morphing is a functionality implemented by madminer that allows for the fast and exact interpolation of event weights (or any derived quantity) from a set of original benchmark points (the “morphing basis”) to anywhere in theory parameter space. In this jupyter notebook, we will be focus on how to utilizing the morphing function of madminer.

In [1]:
# Preparation
import numpy as np

from madminer.utils import morphing as m

The current morphing function can take in two different format of inputs. One format is setting basis points default as work for both and production and decay, with the basis points input shape of (n_basis_points, n_parameters); The other allow user to specify couplings groups of production (gp), decay(gd) and same (gs, work as both production and decay) separately, each with a shape of (n_parameters, n_basis_points). Both ways support overdetermine morphing, which allows user to input as many basis_points as possible for better results. Both methods should output the same matrix and weights values. 

First Input Format

In [2]:
# Example of using the first input format that default couplings that work for both decay and production couplings.

# First, we should create a new morpher object using the code below
morpher_1 = m.PhysicsMorpher(parameter_max_power=[2, 2])

# Components is the power of each parameter in the given coupling combination, for example, if the equation is gs_0^2 * gs_1^3, the components for this combination is [2,3]
# if there's no known components, we can use the function find_components() to find the correspondig components based of the number of parameter of basis, which equals len(parameter_max_power)
components = morpher_1.find_components()
# print("Generated Components: \n", components)

# if we have power of components in hand, we can use morpher.set_components() to set the nubmer of components
this_components = np.array([[4, 0], [3, 1], [2, 2], [1, 3], [0, 4]])
morpher_1.set_components(this_components)
# print("Input Components:\n", morpher.components)

# In this way, the morpher's component is set, then we can set basis, the morpher.set_basis() need to be called after the components is set
# the morpher.set_basis will also call the calculate_morphing matrix method and the morphing matrix will be calculated.
basis_numpy = np.array([[1, -5], [1, -4], [1, -3], [1, -2], [1, -1]])
morpher_1.set_basis(basis_numpy=basis_numpy)
# print("basis:\n", morpher.basis)

# To this step, the prepartion works are done.

In [3]:
# calculate W_i and Neff/xsec with W_i = w_i*sigma_i and Neff = sum(W_i)
def calculate_predict_xsec(xsec, morphing_weights):
    index = len(morphing_weights)
    if len(xsec) < index:
        raise Exception("The number of xsec values is smaller than the number of morphing weights")

    # Get the corresponding xsec values for the morphing weights
    this_xsec = xsec[:index]
    W_i = np.multiply(this_xsec, morphing_weights, dtype=np.float32)
    return sum(W_i)

In [4]:
# Then we can calculate the morphing matrix and corresponding weights with the information we input above.
morphing_1_matrix = morpher_1.calculate_morphing_matrix()
# print("Morphing matrix: \n", morphing_matrix)

# in the parameter input we had input basis points such from (1,-5) to (1,-1), if we want to predict the point theta = [1,1], and the known xsec is listed below
# The coressponding xsec for the above basis points are xsec_simulated[0:5], and we are predicting the value of xsec_simulatd[6]
xsec_simulated = np.array([0.759, 0.53, 0.4, 0.335, 0.316, 0.316, 0.328, 0.34, 0.354, 0.364])
morphing_1_weights = morpher_1.calculate_morphing_weights(theta=[1, 1])
# print("Morphing matrix weights: \n", morphing_weights)

n_base_5 = calculate_predict_xsec(morphing_weights=morphing_1_weights, xsec=xsec_simulated)
print("n_base_5 predict xsec value: \n", n_base_5)
print("simulated value: \n", xsec_simulated[6])
print("\n")

# We can see and compare the predict value with the simulated values.

n_base_5 predict xsec value: 
 0.41500043869018555
simulated value: 
 0.328




Second Input Fromat

The second input format allows you to specify basis from different couplings separately, which allows user to specify couplings groups of production (gp), decay(gd) and same (gs, work as both production and decay) separately, each with a shape of (n_parameters, n_basis_points). In addition, the second input format supports overdetermined morphing, which allows you to input more basis points than minimun requirement for better accruacy. 

In [5]:
# Example of the second input format that allows user to specify different coupling groups.
# We will show the overdetermined functionality and compare with the previous result first.

# First, we should create a new morpher object using the code below
morpher_2 = m.PhysicsMorpher(parameter_max_power=[2, 2])

# We set the componet value the same as previous
this_components = np.array([[4, 0], [3, 1], [2, 2], [1, 3], [0, 4]])
morpher_2.set_components(this_components)

# gs means the couplings that work for both production and decay.
# Please note that the input format of the each coupling group is the transpose of the previous format
# In this case, the gs will have six basis points instead of the previous 5
gs = np.array([[1, 1, 1, 1, 1, 1], [-5, -4, -3, -2, -1, 0]])

# We set the basis value of the morpher
morpher_2.set_basis(basis_d=None, basis_p=None, basis_s=gs)

# To this step, the prepartion works are done.

In [6]:
# The other functions are the is the same as the previous example
# We can get the morphing functions, weights, and xsec values
morpher_2_matrix = morpher_2.calculate_morphing_matrix()

# We input the same theta value as the previous example
morpher_2_weight = morpher_2.calculate_morphing_weights(theta=[1, 1])

# calculate the predict value of point [1,1] with six basis points
n_base_6 = calculate_predict_xsec(morphing_weights=morpher_2_weight, xsec=xsec_simulated)

# We compare the results of the regular and overdetermined morphing with simulated value

print("simulated value:", xsec_simulated[6])
print("regular predict value", n_base_5)
print("overdetermined predict value:", n_base_6)

# We can see with only one additional basis point, the predict value is much more close to the simulated value

simulated value: 0.328
regular predict value 0.41500043869018555
overdetermined predict value: 0.3191666603088379


Below is a table showing the results with different number of basis points(5/7) of the example above. We can see that the more points and the corresponding points are provided, the better prediction results we will get from the morphing. 



In [7]:
"""
+----+-----------+---------------------+---------------------+
| g2 | simulated |      n_base = 5     |      n_base = 7     |
+----+-----------+---------------------+---------------------+
| -5 |   0.759   |  0.7590000033377339 |  0.7590043565724045 |
| -4 |    0.53   |  0.5299999713898771 |  0.5300930421799421 |
| -3 |    0.4    | 0.40000000596042407 | 0.39946969877928495 |
| -2 |   0.335   | 0.33500000834464144 |  0.3361039091832936 |
| -1 |   0.316   |  0.3160000145435327 | 0.31487447768449783 |
| 0  |   0.316   | 0.33900004625320435 |  0.3165692798793316 |
| 1  |   0.328   | 0.41500043869018555 | 0.32788529992103577 |
| 2  |    0.34   |  0.5700044631958008 | 0.34142857789993286 |
| 3  |   0.354   |  0.8450050354003906 | 0.35571449995040894 |
| 4  |   0.364   |  1.2960090637207031 |  0.3751673698425293 |
| 5  |   0.376   |  1.9940032958984375 |  0.4101197123527527 |
| 7  |   0.4205  |   4.49005126953125  |  0.5974075496196747 |
| 9  |   0.5347  |  9.201080322265625  |  1.1184179782867432 |
| 11 |   0.7822  |  17.23541259765625  |  2.2685461044311523 |
| 13 |   1.244   |  29.94012451171875  |  4.4378204345703125 |
+----+-----------+---------------------+---------------------+
"""
print()

Specifying different coupling groups independently. 

We mentioned that the second format allows user to specify different coupling groups separately, which allow user to do morphing with more specified basis points selectoins.
Below is an example using this functinality. 

In [8]:
# Suppose we want to set basis with 1 decay coupling and 3 production couplings

# In the order of gd, gp, gc, the code will determine the number of each coupling parameter based on gd, gp, gc...
n_d = 3
n_p = 1
n_s = 0

# specify gd, gp, gc separately
gd = np.array(
    [
        [0.7071, 0.7071, 0.7071, 0.7071, 0.7071, 0.7071],
        [0, 4.2426, 0, 4.2426, -4.2426, 0],
        [0, 0, 4.2426, 4.2426, 0, -4.2426],
    ]
)
gp = np.array([[0.7071, 0.7071, 0.7071, 0.7071, 0.7071, 0.7071]])
gs = None  # np.array([[1,1,1,1,1], [-5, -4, -3, -2, -1]])

# The known xsec values, we can use the first six for inputs and the rest for validating results
simulate_xsec = np.array(
    [0.515, 0.732, 0.527, 0.742, 0.354, 0.527, 0.364, 0.742, 0.364, 0.621, 0.432, 0.621, 0.432]
)

# We create a new morpher object
morpher_3 = m.PhysicsMorpher(parameter_max_power=[4, 4])

# Please note that we can also set the BSM_max_power in find_components in this input format, which we will disregard the component powers that exceed the limit.
# BSM couplings are couplings that are not gd_0, gp_0, gs_0.
# In this case as Nd = 1, Np=3, components[0] is gd_0, and components[1] is gp_1, this will eliminate the groups that the rest values exceed 1
components_pmax = morpher_3.find_components(Nd=n_d, Np=n_p, Ns=n_s, BSM_max_power=1)
# print("Components with BSM max power restriction: ", components_pmax)

# For this example we will not set the BSM_max limit, you can check and compare the difference with the print statement below.
components = morpher_3.find_components(Nd=n_d, Np=n_p, Ns=n_s)
# print("Non pmax components", components)

# We set the basis and calculate the matrix
# We set the known first six basis points as the inputs(gd and gp)
morpher_3.set_basis(basis_d=gd, basis_p=gp, basis_s=gs)
morpher_3.calculate_morphing_matrix()

# The prepartion steps are done

array([[ 4.00015344e+00,  1.73472348e-18, -8.67361738e-19,
         1.73472348e-18,  2.16840434e-19, -4.33680869e-19],
       [ 1.11022302e-16,  3.33346120e-01,  2.21710443e-17,
         1.93280886e-19, -3.33346120e-01, -8.01533606e-17],
       [-1.14491749e-16, -4.48656738e-17,  3.33346120e-01,
         1.44208707e-18, -4.12791158e-17, -3.33346120e-01],
       [-1.11115373e-01,  5.55576867e-02,  4.15495976e-16,
        -3.33916403e-17,  5.55576867e-02, -3.60505754e-16],
       [ 1.11115373e-01, -1.11115373e-01, -1.11115373e-01,
         1.11115373e-01, -8.24196939e-17, -5.41626748e-17],
       [-1.11115373e-01, -3.26528840e-16,  5.55576867e-02,
        -3.78513043e-18,  3.79226289e-16,  5.55576867e-02]])

In [9]:
# Then we calculate the morphing weights, we can use to predict the last known xsec value
# We input the basis points positions we want to calculate the xsec_value, which for the last known xsec value is input as below
# theta_d = [d_0, d_1, d_2], theta_p = [p_0]
morpher_3_weights = morpher_3.calculate_morphing_weights(
    theta_d=[0.7071, -3 * 0.7071, 3 * 0.7071], theta_p=[0.7071]
)

morpher_3_predicted_xsec = calculate_predict_xsec(
    xsec=simulate_xsec, morphing_weights=morpher_3_weights
)

# The expected xsec value is the the last one in simulated_xsec, which is the 13th input we have
expected_xsec_value = simulate_xsec[-1]

print("Predict xsec value: ", morpher_3_predicted_xsec)
print("Simulated xsec value", expected_xsec_value)

Predict xsec value:  0.4310000017285347
Simulated xsec value 0.432


We can see that we are able to predict the xsec value that is very close to the actual simulated xsec values. Remember, we only used the first the six xsec values and points to predict the 13th xsec value. As mentioned earlier, we supports overdetermined morphing and can always input more known basis points and corresponding xsec values to get better results. 